In [4]:
import os
os.chdir("/home/user/Documents/projects/openclip")
# os.chdir("/home/zhuoyan/vision/openclip")
import numpy as np
from collections import defaultdict 

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
data_path = "/home/user/Documents/projects/openclip/log"
# data_path = "/home/zhuoyan/vision/openclip/log"
import json
import time 

# Function to load JSON data from a file
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

In [5]:

data = load_json(f"{data_path}/baseline_results_128branch_10SamplePerClass.json")
df = pd.DataFrame(data)
df.rename(columns={'imagenet-zeroshot-val-top1': 'accs'}, inplace=True)

In [6]:
df = df.sort_values(by='macs', ascending=True)
# Using Seaborn to plot
sns.scatterplot(data=df, x='macs', y='accs', color='blue', s=40)

plt.xlim(0, 1.1)
plt.ylim(-0.1, 1.1)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.title('Scatter Plot of accs vs macs')
plt.show()

In [7]:
# Define the range width for 'macs', e.g., 0.05 for 5% intervals
bin_width = 0.01
max_points = []

# Iterate over the 'macs' range using the bin size
for bin_start in np.arange(0, 1, bin_width):
    # Find all points within the current bin range
    
    bin_end = bin_start + bin_width
    # print(bin_start, bin_end)
    bin_df = df[(df['macs'] >= bin_start) & (df['macs'] <= bin_end)]
    
    # If there are points in the bin, select the one with the maximum 'accs'
    if not bin_df.empty:
        max_point = bin_df.loc[bin_df['accs'].idxmax()]
        max_points.append(max_point)

saved_points = []
for item in max_points:
    point_dict = item.to_dict()  # Convert Series to dict
    point_dict['Name'] = item.name  # Add the 'Name' which is the index of the Series
    saved_points.append(point_dict)


In [8]:
up = [0,128]  ## saved the acc and branch name
tabled_points = []
for dic in saved_points:
    if dic['accs'] >= up[0]:
        up[0] = dic['accs']
        up[1] = dic['Name']
        tabled_points.append(dic)
    else:
        dic['accs']= up[0]

In [9]:
df

accs  imagenet-zeroshot-val-top5      macs
31  0.0020                      0.0072  0.188495
74  0.0010                      0.0056  0.188495
66  0.0012                      0.0059  0.269646
25  0.0010                      0.0057  0.269646
67  0.0015                      0.0054  0.269646
..     ...                         ...       ...
65  0.1947                      0.4176  0.837699
60  0.1989                      0.4095  0.837699
35  0.4074                      0.7007  0.837699
26  0.2883                      0.5416  0.837699
0   0.6261                      0.8745  1.000000

[129 rows x 3 columns]

In [10]:
# Convert the list of points to a DataFrame
upper_curve_df = pd.DataFrame(saved_points)

# Using Seaborn to plot the scatter plot
sns.scatterplot(data=df, x='macs', y='accs',  s=40)

# Plotting the manually selected upper curve
plt.plot(upper_curve_df['macs'], upper_curve_df['accs'], color='red', label='Upper Curve', marker='o')

# Set the limits, grid, and title as before
plt.xlim(0, 1.1)
plt.ylim(-0.1, 1.1)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.title('Scatter Plot of accs vs macs')

# Add the legend with the location set to bottom right
plt.legend(loc='lower right')

# Display the plot
plt.show()